In [1]:
import torch 
import torch.nn as nn
import os
import argparse
from torchvision import datasets, transforms
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import sys
from PIL import Image
import requests
from io import BytesIO
import urllib.request as url_req
import pickle
from Model import get_model
from utils import *
import json
from model_and_data import Data
from FGSM import FGSM
from BIM import BIM
from visualize import visualise

In [2]:
device = torch.device('cpu')

In [3]:
model = get_model(device)                  # loads a pretrained vgg11 model
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [4]:
class CarliniWagnerL2():
    # https://github.com/rwightman/pytorch-nips2017-attack-example/blob/master/attacks/attack_carlini_wagner_l2.py
    def __init__(self, orig_img, model, target, initial_constt, clip_min, clip_max, confidence_param, 
                 binary_search_steps, max_steps):
        self.model = model
        self.clip_min = clip_min
        self.clip_max = clip_max
        self.confidence= confidence_param
        self.bsearch_steps = binary_search_steps
        self.num_classes = 1000
        self.target = target          # tensor
        self.initial_constt = initial_constt
        self.repeat = binary_search_steps >= 10
        self.orig_img = orig_img.clone()
        self.max_steps = max_steps
        
    def _compare(self, output, target):
        if not isinstance(output, (float, int, np.int64)):
            output = np.copy(output)
            output[target] -= self.confidence
            output = np.argmax(output)
        
        return output == target

    
    def _loss(self,output, target, dist, scale_constt):
        # compute probability of the label class versus the maximum other
        real = (target * output.data).sum(1)
        other = ((1 - target) * output.data - target*10000).max(1)[0]   # target is one-hot encoded and 
                                                    # gives max{Z(x')<subs>i  where i!=t (t is target class)} 
            
        loss1 = torch.clamp(other - real + self.confidence, min = 0.) 
        loss1 = torch.sum(scale_constt*loss1)
        
        loss2 = dist.sum()
        loss = loss1+loss2
        print(loss1)
        print(loss2)
        print('Loss =',loss)
        return loss
    
    def _optimize(self, optimizer, model, input_var, modifier_var, target_var, scale_const_var, input_orig=None):
        input_adv = tanh_rescale(modifier_var+input_var, self.clip_min,self.clip_max)
        
        output = model(input_adv)
        
        if input_orig is None:
            dist = l2_dist(input_adv, input_var,keepdim=False)
            print('hi')
        else:
            dist = l2_dist(input_adv, input_orig, keepdim=False)
            print(input_adv.shape)
            print(input_orig.shape)
            print((input_orig))
            print('No')
            
        loss = self._loss(output, target_var, dist, scale_const_var)
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        
        loss_np = loss.data[0]
        print(loss_np)
        dist_np = dist.data.cpu().numpy()
        output_np = output.data.cpu().numpy()
        input_adv_np = input_adv.data.permute(0, 2, 3, 1).cpu().numpy()  # back to BHWC for numpy consumption
        return loss_np, dist_np, output_np, input_adv_np
    
    def run(self, model, inputs,target, batch_idx=0):
        batch_size = inputs.size(0)
        
        # set the lower and upper bounds accordingly
        lower_bound = np.zeros(batch_size)
        scale_const = np.ones(batch_size) * self.initial_constt
        upper_bound = np.ones(batch_size) * 1e10
        
        # python/numpy placeholders for the overall best l2, label score, and adversarial image
        o_best_l2 = [1e10] * batch_size
        o_best_score = [-1] * batch_size
        o_best_attack = inputs.permute(0, 2, 3, 1).cpu().detach().numpy()
        
        input_var = torch.tensor(torch_arctanh(inputs),requires_grad=False)
        input_orig = tanh_rescale(input_var, self.clip_min, self.clip_max)
        
        target_onehot = torch.zeros(target.size() + (self.num_classes,))
#         target_onehot = target_onehot.cuda()
        target_onehot.scatter_(1, target.unsqueeze(1), 1.)
        target_var = torch.tensor(target_onehot, requires_grad=False)
        
        modifier = torch.zeros(input_var.size()).float()
        
        modifier = torch.normal(mean=modifier, std = 1e-3)
#         modifier = modifier.cuda()
        modifier_var = torch.tensor(modifier, requires_grad=True)
        
        optimizer = optim.Adam([modifier_var], lr=5e-4)
        
        
        for search_step  in range(self.bsearch_steps):
            for i, x in enumerate(scale_const):
                print(i, x)
            best_l2 = [1e10] * batch_size
            best_score = [-1] * batch_size
            
            if self.repeat and search_step == self.binary_search_steps - 1:
                scale_const = upper_bound
            
            scale_const_tensor = torch.from_numpy(scale_const).float()
#             scale_const_tensor = scale_const_tensor.cuda()
            scale_const_var = torch.tensor(scale_const_tensor,requires_grad=False)
        
            prev_loss = 1e6
            for step in range(self.max_steps):
                loss, dist, output, adv_img = self._optimize(
                    optimizer,
                    model,
                    input_var,
                    modifier_var,
                    target_var,
                    scale_const_var,
                    input_orig)
                if step % 100 == 0 or step == self.max_steps - 1:
                    print('Step: {0:>4}, loss: {1:6.4f}, dist: {2:8.5f}, modifier mean: {3:.5e}'.format(
                        step, loss, dist.mean(), modifier_var.data.mean()))
                    
                                 # update best result found
                for i in range(batch_size):
                    target_label = target[i]
                    output_logits = output[i]
                    output_label = np.argmax(output_logits)
                    di = dist[i]
                    if step % 100 == 0:
                            print('{0:>2} dist: {1:.5f}, output: {2:>3}, {3:5.3}, target {4:>3}'.format(
                                i, di, output_label, output_logits[output_label], target_label))   
                
                    if di < best_l2[i] and self._compare(output_logits, target_label):
                
                        best_l2[i] = di
                        best_score[i] = output_label
                    if di < o_best_l2[i] and self._compare(output_logits, target_label):
                        o_best_l2[i] = di
                        o_best_score[i] = output_label
                        o_best_attack[i] = adv_img[i]
        
            batch_failure = 0
            batch_success = 0
            for i in range(batch_size):
                if self._compare(best_score[i], target[i]) and best_score[i] != -1:
                    # successful, do binary search and divide const by two
                    upper_bound[i] = min(upper_bound[i], scale_const[i])
                    if upper_bound[i] < 1e9:
                        scale_const[i] = (lower_bound[i] + upper_bound[i]) / 2
#                     if self.debug:
#                         print('{0:>2} successful attack, lowering const to {1:.3f}'.format(
#                             i, scale_const[i]))
                else:
                    # failure, multiply by 10 if no solution found
                    # or do binary search with the known upper bound
                    lower_bound[i] = max(lower_bound[i], scale_const[i])
                    if upper_bound[i] < 1e9:
                        scale_const[i] = (lower_bound[i] + upper_bound[i]) / 2
                    else:
                        scale_const[i] *= 10
                if self._compare(o_best_score[i], target[i]) and o_best_score[i] != -1:
                    batch_success += 1
                else:
                    batch_failure += 1

            print('Num failures: {0:2d}, num successes: {1:2d}\n'.format(batch_failure, batch_success))
            sys.stdout.flush()
            # end outer search loop

        return o_best_attack

In [5]:
imgs = os.listdir('imagenet_imgs/')
batch_size = len(os.listdir('imagenet_imgs/'))
for idx,img_name in enumerate(imgs):
#     print(idx)
    if idx==2:
        img_path = os.path.join('imagenet_imgs/',img_name)
        data = Data(model,device, None,None)
        img_tsor = data.preprocess_data(Image.open(img_path))
        #         imshow(img_tsor,'dgs')
        img_tsor.unsqueeze_(0)
        img_tsor = img_tsor.to(device)
        img_tsor.requires_grad_(True)

        label = img_name.split('_')[0]
        label = torch.tensor([int(label)],requires_grad=False)
        label = label.to(device)
        #       
        print(label.shape)

#         criterion = nn.CrossEntropyLoss()
        ############ Unperturbed Model ######################
        unpert_output,unpert_pred, unpert_op_probs, unpert_pred_prob = getPredictionInfo(model,img_tsor)
        targetLabel = torch.tensor([300],requires_grad=False).to(device)
        wagner = CarliniWagnerL2(img_tsor, model, targetLabel, 0.1, -1, 1, 0, 4,800)
        wagner_attack = wagner.run(model,img_tsor,targetLabel)


torch.Size([1])
0 0.1
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([[[[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.3452,  0.3803,  0.3627,  ...,     nan,     nan,     nan],
          [ 0.0826,  0.2227,  0.2402,  ...,     nan,     nan,     nan],
          [ 0.0126,  0.0126, -0.0049,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.4091,  0

/home/anishmadan23/.local/lib/python3.5/site-packages/ipykernel_launcher.py:63: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([[[[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.3452,  0.3803,  0.3627,  ...,     nan,     nan,     nan],
          [ 0.0826,  0.2227,  0.2402,  ...,     nan,     nan,     nan],
          [ 0.0126,  0.0126, -0.0049,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.4091,  0.3742,  0.3742,  ..., 

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([[[[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.3452,  0.3803,  0.3627,  ...,     nan,     nan,     nan],
          [ 0.0826,  0.2227,  0.2402,  ...,     nan,     nan,     nan],
          [ 0.0126,  0.0126, -0.0049,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.4091,  0.3742,  0.3742,  ..., 

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([[[[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.3452,  0.3803,  0.3627,  ...,     nan,     nan,     nan],
          [ 0.0826,  0.2227,  0.2402,  ...,     nan,     nan,     nan],
          [ 0.0126,  0.0126, -0.0049,  ...,     nan,     nan,     nan],
          ...,
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
          [    nan,     nan,     nan,  ...,     nan,     nan,     nan]],

         [[ 0.4091,  0.3742,  0.3742,  ..., 

KeyboardInterrupt: 

In [ ]:
def show_results():
    global model
    model_misclassifns = 0
    criterion = nn.CrossEntropyLoss()
    misclassfns = 0
    total = 0
    wagner_att = None
    eps = 0.1
    eps_iter = 0.04
    k=0
    k_img = None
    for i,data in enumerate(testset):
        if i<10:
#             print(i)
            images_, labels_ = data
            images, labels = images_, labels_

            for img , label in zip(images, labels):
                img = img.unsqueeze(0)      # to match dimensions when we input single image instead of mini-batch
                img = img.to(device)
                img.requires_grad_(True)
                    
                print(label)
                label = label.unsqueeze(0)
                
                label = label.to(device)

                unpert_predictedLabel, unpert_pred_prob = unperturbed_model(model,img,label)
                targetLabel = torch.tensor([4],requires_grad=False).to(device)
#                 lbfgs_att = LBFGS_Attack(model,criterion,img,label,unpert_predictedLabel,targetLabel,5,500,0,1,0.01,device)
#                 adv_img = lbfgs_att.attack(img,targetLabel)

#                 if i==3:
#                     k+=2
#                     if k==2:
# #                         wagner = CarliniWagnerL2(img, model, targetLabel, 0.1, -1, 1, 0, 4,800)
# #                         wagner_attack = wagner.run(model,img,targetLabel)
# #                         wagner_att = wagner_attack
# #                         k_img = img 
#                         pgd = PGDAttack(model,img,label,0.01,0.3,device)
#                         advImg,losses = pgd.attack(30)
#                         k_img = img
                
    print(total,misclassfns)
    return img,label,adv_img,perturbation, pred_adv, adv_pred_prob
#     return wagner_att,k_img
#     print(total,success)